In [11]:
import pandas as pd
import re
from evaluate import load

# 데이터 로드
submission = pd.read_csv('./prediction/submit_solar_v3.csv')
dev_df = pd.read_csv('./data/dev.csv')

print(f"제출 파일: {len(submission)}개")
print(f"Dev 파일: {len(dev_df)}개")

# ROUGE 메트릭
rouge = load("rouge")

제출 파일: 499개
Dev 파일: 499개


## 1. 현재 제출 파일 분석

In [12]:
# 요약 길이 분석
submission['length'] = submission['summary'].apply(lambda x: len(str(x).split()))

print("📊 현재 제출 파일 통계:")
print(f"  - 평균 길이: {submission['length'].mean():.1f} 단어")
print(f"  - 최소 길이: {submission['length'].min()} 단어")
print(f"  - 최대 길이: {submission['length'].max()} 단어")
print(f"  - 중앙값: {submission['length'].median():.1f} 단어")

# 문제 케이스 찾기
print("\n⚠️ 잠재적 문제 케이스:")

# 1. 따옴표로 시작하는 요약
quoted = submission[submission['summary'].str.startswith('"')]
print(f"  - 따옴표로 감싸진 요약: {len(quoted)}개")

# 2. 문장이 끊긴 것처럼 보이는 케이스 (문장 부호 없이 끝남)
incomplete = submission[~submission['summary'].str.endswith(('.', '!', '?', '다', '요', '습니다'))]
print(f"  - 불완전한 문장: {len(incomplete)}개")

# 3. 너무 긴 요약 (70+ 단어)
too_long = submission[submission['length'] > 70]
print(f"  - 너무 긴 요약 (70+ 단어): {len(too_long)}개")

# 4. 너무 짧은 요약 (15- 단어)
too_short = submission[submission['length'] < 15]
print(f"  - 너무 짧은 요약 (15- 단어): {len(too_short)}개")

📊 현재 제출 파일 통계:
  - 평균 길이: 16.1 단어
  - 최소 길이: 6 단어
  - 최대 길이: 35 단어
  - 중앙값: 14.0 단어

⚠️ 잠재적 문제 케이스:
  - 따옴표로 감싸진 요약: 0개
  - 불완전한 문장: 21개
  - 너무 긴 요약 (70+ 단어): 0개
  - 너무 짧은 요약 (15- 단어): 252개


## 2. 후처리 함수 정의

In [13]:
def postprocess_v1(summary: str) -> str:
    """
    기본 후처리
    - 따옴표 제거
    - 공백 정리
    """
    # 따옴표 제거
    summary = summary.strip('""''""')
    
    # 공백 정리
    summary = re.sub(r'\s+', ' ', summary).strip()
    
    return summary


def postprocess_v2(summary: str) -> str:
    """
    중급 후처리
    - v1 기능 포함
    - 불완전한 문장 정리
    - 마지막 문장이 중간에 끊긴 경우 제거
    """
    summary = postprocess_v1(summary)
    
    # 마지막 문장이 불완전하면 제거 (마지막 완전한 문장까지만 유지)
    sentences = re.split(r'([.!?]\s+)', summary)
    
    # 문장과 구분자를 다시 합침
    complete_text = ""
    for i in range(0, len(sentences) - 1, 2):
        if i + 1 < len(sentences):
            complete_text += sentences[i] + sentences[i + 1]
    
    # 마지막 부분이 완전한 문장으로 끝나면 추가
    if sentences and sentences[-1].strip() and sentences[-1].strip()[-1] in '.!?다요':
        complete_text += sentences[-1]
    
    return complete_text.strip()


def postprocess_v3(summary: str, max_sentences: int = 3) -> str:
    """
    고급 후처리
    - v2 기능 포함
    - 최대 문장 수 제한 (일반적으로 2-3문장이 최적)
    """
    summary = postprocess_v2(summary)
    
    # 문장 분리
    sentences = re.split(r'(?<=[.!?])\s+', summary)
    
    # 최대 문장 수로 제한
    if len(sentences) > max_sentences:
        sentences = sentences[:max_sentences]
    
    return ' '.join(sentences).strip()


def postprocess_v4(summary: str, max_sentences: int = 3) -> str:
    """
    최종 후처리
    - v3 기능 포함
    - 특수토큰 정규화 (#Person1# vs #Person1#)
    - 중복 공백 제거
    """
    summary = postprocess_v3(summary, max_sentences)
    
    # 특수토큰 정규화 (일관성 확보)
    summary = re.sub(r'#Person(\d+)#', r'#Person\1#', summary)
    summary = re.sub(r'#PhoneNumber#', '#PhoneNumber#', summary)
    
    # 최종 공백 정리
    summary = re.sub(r'\s+', ' ', summary).strip()
    
    return summary


print("✅ 4개의 후처리 함수 정의 완료")

✅ 4개의 후처리 함수 정의 완료


## 3. Dev 셋에서 후처리 성능 비교

In [14]:
# Dev 셋에 대한 예측이 필요하므로 우선 Test 셋에 적용한 결과로 대략적인 영향을 확인
# 실제로는 Dev 셋에 대한 추론 결과가 있어야 정확한 평가 가능

print("⚠️ 주의: Dev 셋 추론 결과가 없어 Test 셋으로 후처리 영향 확인")
print("실제 점수 변화는 리더보드 제출 후 확인 필요\n")

# 각 버전별 후처리 적용
versions = {
    'original': submission['summary'].tolist(),
    'v1_basic': submission['summary'].apply(postprocess_v1).tolist(),
    'v2_complete': submission['summary'].apply(postprocess_v2).tolist(),
    'v3_limit3': submission['summary'].apply(lambda x: postprocess_v3(x, max_sentences=3)).tolist(),
    'v3_limit2': submission['summary'].apply(lambda x: postprocess_v3(x, max_sentences=2)).tolist(),
    'v4_final3': submission['summary'].apply(lambda x: postprocess_v4(x, max_sentences=3)).tolist(),
    'v4_final2': submission['summary'].apply(lambda x: postprocess_v4(x, max_sentences=2)).tolist(),
}

# 통계 비교
print("📊 버전별 통계 비교:\n")
for name, summaries in versions.items():
    lengths = [len(s.split()) for s in summaries]
    avg_len = sum(lengths) / len(lengths)
    
    # 변화량 계산 (original 대비)
    if name == 'original':
        print(f"  [{name:15s}] 평균: {avg_len:5.1f} 단어")
    else:
        orig_lengths = [len(s.split()) for s in versions['original']]
        orig_avg = sum(orig_lengths) / len(orig_lengths)
        diff = avg_len - orig_avg
        print(f"  [{name:15s}] 평균: {avg_len:5.1f} 단어 ({diff:+5.1f})")

⚠️ 주의: Dev 셋 추론 결과가 없어 Test 셋으로 후처리 영향 확인
실제 점수 변화는 리더보드 제출 후 확인 필요

📊 버전별 통계 비교:

  [original       ] 평균:  16.1 단어
  [v1_basic       ] 평균:  16.1 단어 ( +0.0)
  [v2_complete    ] 평균:  15.8 단어 ( -0.3)
  [v3_limit3      ] 평균:  15.6 단어 ( -0.5)
  [v3_limit2      ] 평균:  14.4 단어 ( -1.8)
  [v4_final3      ] 평균:  15.6 단어 ( -0.5)
  [v4_final2      ] 평균:  14.4 단어 ( -1.8)


## 4. 샘플 비교

In [15]:
# 문제가 있었던 케이스들을 후처리 버전별로 비교
problem_indices = [0, 2, 6, 41]  # 따옴표, 불완전한 문장 등

print("🔍 문제 케이스 후처리 비교:\n")
for idx in problem_indices:
    print(f"\n{'='*80}")
    print(f"[{submission.iloc[idx]['fname']}]")
    print(f"{'='*80}")
    
    print(f"\n[Original ({len(versions['original'][idx].split())} 단어)]")
    print(versions['original'][idx])
    
    print(f"\n[V4 - Final (3문장) ({len(versions['v4_final3'][idx].split())} 단어)]")
    print(versions['v4_final3'][idx])
    
    print(f"\n[V4 - Final (2문장) ({len(versions['v4_final2'][idx].split())} 단어)]")
    print(versions['v4_final2'][idx])

🔍 문제 케이스 후처리 비교:


[test_0]

[Original (34 단어)]
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다. 이 정책은 모든 통신에 적용되며, 즉시 메시지 사용은 금지됩니다. Ms. Dawson은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용한다고 말하지만, #Person1#은 이를 원하지 않습니다.

[V4 - Final (3문장) (20 단어)]
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다. 이 정책은 모든 통신에 적용되며, 즉시 메시지 사용은 금지됩니다.

[V4 - Final (2문장) (11 단어)]
#Person1#은 Ms. Dawson에게 모든 직원에게 사내 메모로 보내야 할 정책을 요청합니다.

[test_2]

[Original (12 단어)]
#Person1#은 Kate에게 Masha와 Hero의 이혼 소식을 전하며, 양육권은 Masha가 가질 예정이라고 말합니다.

[V4 - Final (3문장) (12 단어)]
#Person1#은 Kate에게 Masha와 Hero의 이혼 소식을 전하며, 양육권은 Masha가 가질 예정이라고 말합니다.

[V4 - Final (2문장) (12 단어)]
#Person1#은 Kate에게 Masha와 Hero의 이혼 소식을 전하며, 양육권은 Masha가 가질 예정이라고 말합니다.

[test_6]

[Original (14 단어)]
#Person1#은 #Person2#가 수두에 걸린 것 같다고 생각하지만, #Person2#는 그냥 발진이나 알레르기일 수도 있다고 말합니다.

[V4 - Final (3문장) (14 단어)]
#Person1#은 #Person2#가 수두에 걸린 것 같다고 생각하지만, #Person2#는 그냥 발진이나 알레르기일 수도 있다고 말합니다.

[V4 - Final (2문장) (14 단어)]
#Person1#은 #Person2#가 수두에 걸린 것 같다고 생각하지

## 5. 최적 후처리 선택 및 새 제출 파일 생성

In [6]:
# 권장: v4_final3 (최대 3문장 제한)
# 이유: 
# 1. 따옴표 제거로 깔끔한 출력
# 2. 불완전한 문장 제거로 품질 향상
# 3. 3문장 제한으로 적절한 길이 유지 (Dev 평균 참고시)

print("🎯 권장 후처리 버전: v4_final3 (최대 3문장)")
print("\n이유:")
print("  1. 따옴표 제거 → 깔끔한 출력")
print("  2. 불완전한 문장 제거 → 품질 향상")
print("  3. 3문장 제한 → ROUGE 최적화")
print("  4. 특수토큰 정규화 → 일관성 확보")

# 새 제출 파일 생성
new_submission = submission[['fname']].copy()
new_submission['summary'] = versions['v4_final3']

output_path = './prediction/submit_solar_v3_postprocessed.csv'
new_submission.to_csv(output_path, index=False)

print(f"\n✅ 새 제출 파일 저장: {output_path}")
print(f"   총 {len(new_submission)}개 요약")

🎯 권장 후처리 버전: v4_final3 (최대 3문장)

이유:
  1. 따옴표 제거 → 깔끔한 출력
  2. 불완전한 문장 제거 → 품질 향상
  3. 3문장 제한 → ROUGE 최적화
  4. 특수토큰 정규화 → 일관성 확보

✅ 새 제출 파일 저장: ./prediction/submit_solar_v3_postprocessed.csv
   총 499개 요약


## 6. (선택) 2문장 버전도 생성

In [8]:
# 만약 3문장 버전이 안 좋다면 2문장 버전도 시도 가능
new_submission_v2 = submission[['fname']].copy()
new_submission_v2['summary'] = versions['v4_final2']

output_path_v2 = './prediction/submit_solar_v3_postprocessed_2sent.csv'
new_submission_v2.to_csv(output_path_v2, index=False)

print(f"✅ 2문장 제한 버전 저장: {output_path_v2}")
print(f"   총 {len(new_submission_v2)}개 요약")

✅ 2문장 제한 버전 저장: ./prediction/submit_solar_v3_postprocessed_2sent.csv
   총 499개 요약


## 7. 결과 요약

### 생성된 파일:
1. `submit_solar_v3_postprocessed.csv` - **권장** (3문장 제한)
2. `submit_solar_v3_postprocessed_2sent.csv` - 대안 (2문장 제한)

### 예상 효과:
- ✅ 따옴표 제거로 ROUGE-L 소폭 향상 예상
- ✅ 불완전한 문장 제거로 ROUGE-2 향상 예상  
- ✅ 문장 수 최적화로 전반적인 점수 향상 예상
- 📈 예상 점수 변화: **51.77 → 51.85~52.0** (0.08~0.23점 상승)

### 다음 단계:
1. `submit_solar_v3_postprocessed.csv`를 리더보드에 제출
2. 점수 확인
3. 만약 점수가 떨어지면 `submit_solar_v3_postprocessed_2sent.csv` 시도
4. 점수가 오르면 성공! 🎉

In [16]:
# 신규 제출 파일 생성
print("🎯 신규 제출 파일 생성 순서:")
print("="*80)

# 1. v5 - 공격적 2문장 버전 (권장)
submission_v5_2sent = submission_orig[['fname']].copy()
submission_v5_2sent['summary'] = new_versions['v5_aggressive_2sent']
output_v5_2sent = './prediction/submit_solar_v4_aggressive_2sent.csv'
submission_v5_2sent.to_csv(output_v5_2sent, index=False)
avg_len_v5_2 = submission_v5_2sent['summary'].apply(lambda x: len(x.split())).mean()
print(f"\n✅ [1] {output_v5_2sent}")
print(f"   평균 길이: {avg_len_v5_2:.1f} 단어")
print(f"   예상 점수: 51.95~52.05")
print(f"   전략: 3문장 → 2문장 + 접속사 제거")

# 2. v5 - 공격적 1문장 버전 (고위험-고수익)
submission_v5_1sent = submission_orig[['fname']].copy()
submission_v5_1sent['summary'] = new_versions['v5_aggressive_1sent']
output_v5_1sent = './prediction/submit_solar_v4_aggressive_1sent.csv'
submission_v5_1sent.to_csv(output_v5_1sent, index=False)
avg_len_v5_1 = submission_v5_1sent['summary'].apply(lambda x: len(x.split())).mean()
print(f"\n✅ [2] {output_v5_1sent}")
print(f"   평균 길이: {avg_len_v5_1:.1f} 단어")
print(f"   예상 점수: 51.90~52.10 (변동성 큼)")
print(f"   전략: 1문장으로 극한 압축")

# 3. v6 - 초공격적 버전 (실험용)
submission_v6 = submission_orig[['fname']].copy()
submission_v6['summary'] = new_versions['v6_ultra']
output_v6 = './prediction/submit_solar_v5_ultra.csv'
submission_v6.to_csv(output_v6, index=False)
avg_len_v6 = submission_v6['summary'].apply(lambda x: len(x.split())).mean()
print(f"\n✅ [3] {output_v6}")
print(f"   평균 길이: {avg_len_v6:.1f} 단어")
print(f"   예상 점수: 51.85~52.15 (실험용)")
print(f"   전략: 핵심 정보만 극도로 압축")

print("\n" + "="*80)
print("📋 제출 가이드 및 추천 순서:")
print("="*80)
print("🥇 [우선] submit_solar_v4_aggressive_2sent.csv 제출")
print("   → 51.94보다 높으면 성공! (목표: 52.0+)")
print("   → 낮으면 현재 버전(51.94) 유지\n")
print("🥈 [2순위] 점수가 오르면 submit_solar_v4_aggressive_1sent.csv 시도")
print("   → 고위험-고수익 전략\n")
print("🥉 [백업] 모두 안 되면 현재 51.94점 유지 (이미 성공!)")
print("="*80)

🎯 신규 제출 파일 생성 순서:


NameError: name 'submission_orig' is not defined

## 10. 신규 제출 파일 생성 및 제출 가이드

In [ ]:
# 주요 케이스 샘플 비교
print("🔍 주요 케이스 비교:\n")
test_cases = [0, 5, 41]  # 변화가 컸던 케이스들

for idx in test_cases:
    print(f"\n{'='*80}")
    print(f"[{submission_orig.iloc[idx]['fname']}]")
    print(f"{'='*80}")
    
    print(f"\n[원본] ({len(submission_orig.iloc[idx]['summary'].split())} 단어)")
    orig_text = submission_orig.iloc[idx]['summary']
    print(orig_text[:200] + "..." if len(orig_text) > 200 else orig_text)
    
    print(f"\n[v4 - 현재 베스트 51.94] ({len(new_versions['v4_final3_current'][idx].split())} 단어)")
    print(new_versions['v4_final3_current'][idx])
    
    print(f"\n[v5 - 공격적 2문장] ({len(new_versions['v5_aggressive_2sent'][idx].split())} 단어)")
    print(new_versions['v5_aggressive_2sent'][idx])
    
    print(f"\n[v6 - 초공격적] ({len(new_versions['v6_ultra'][idx].split())} 단어)")
    print(new_versions['v6_ultra'][idx])

In [ ]:
def postprocess_v5_aggressive(summary: str, max_sentences: int = 2) -> str:
    """
    공격적 후처리 (v5)
    - v4 기능 포함
    - 2문장으로 더 강하게 제한
    - 불필요한 접속사 제거
    - 중복 표현 통일
    """
    summary = postprocess_v4(summary, max_sentences=max_sentences)
    
    # 불필요한 문장 시작 접속사 제거
    summary = re.sub(r'(?<=\.\s)(그리고|또한|하지만|그러나|그런데)\s+', '', summary)
    
    # 중복 표현 통일 (~라고 말합니다 → ~라고 합니다)
    summary = re.sub(r'라고\s+말합니다', '라고 합니다', summary)
    summary = re.sub(r'이라고\s+말합니다', '이라고 합니다', summary)
    
    # 공백 정리
    summary = re.sub(r'\s+', ' ', summary).strip()
    
    return summary


def postprocess_v6_ultra(summary: str) -> str:
    """
    초공격적 후처리 (v6) - 1.5문장 전략
    - 핵심 문장 1-2개만 유지
    - 긴 복합문을 단문으로 분리
    - 정보 밀도 최대화
    """
    summary = postprocess_v5_aggressive(summary, max_sentences=2)
    
    # 쉼표로 연결된 긴 문장을 분리 (단, 핵심만)
    sentences = re.split(r'(?<=[.!?])\s+', summary)
    
    if len(sentences) > 2:
        # 첫 2문장만
        sentences = sentences[:2]
    elif len(sentences) == 1 and len(sentences[0].split(',')) > 4:
        # 단일 문장이 너무 길면 중요한 부분만
        parts = sentences[0].split(',')
        # 앞의 2-3개 절만 유지
        sentences = [','.join(parts[:3]) + '.']
    
    return ' '.join(sentences).strip()


# 새 버전들 생성
print("🔬 새로운 후처리 버전 생성 중...\n")

new_versions = {
    'v4_final3_current': postprocessed['summary'].tolist(),  # 현재 베스트 (51.94)
    'v5_aggressive_2sent': submission_orig['summary'].apply(lambda x: postprocess_v5_aggressive(x, 2)).tolist(),
    'v5_aggressive_1sent': submission_orig['summary'].apply(lambda x: postprocess_v5_aggressive(x, 1)).tolist(),
    'v6_ultra': submission_orig['summary'].apply(postprocess_v6_ultra).tolist(),
}

# 통계 비교
print("📊 새 버전별 통계:\n")
for name, summaries in new_versions.items():
    lengths = [len(s.split()) for s in summaries]
    avg_len = sum(lengths) / len(lengths)
    print(f"  [{name:25s}] 평균: {avg_len:5.1f} 단어")

print("✅ 새로운 후처리 버전 생성 완료")

## 9. 함수 정의 및 추가 개선 전략

In [ ]:
# 상세 비교 분석 + 원본 재로드
import pandas as pd

# 원본과 후처리 버전 로드
original = pd.read_csv('./prediction/submit_solar_v3.csv')
postprocessed = pd.read_csv('./prediction/submit_solar_v3_postprocessed.csv')
submission_orig = original.copy()  # 이후 함수에서 사용할 원본 보존

# 길이 비교
original['orig_len'] = original['summary'].apply(lambda x: len(str(x).split()))
postprocessed['post_len'] = postprocessed['summary'].apply(lambda x: len(str(x).split()))

# 통계
print("📊 길이 변화 분석:")
print(f"  원본 평균: {original['orig_len'].mean():.1f} 단어")
print(f"  후처리 평균: {postprocessed['post_len'].mean():.1f} 단어")
print(f"  차이: {postprocessed['post_len'].mean() - original['orig_len'].mean():.1f} 단어\n")

# 변화가 큰 케이스 찾기
merged = pd.DataFrame({
    'fname': original['fname'],
    'orig_summary': original['summary'],
    'post_summary': postprocessed['summary'],
    'orig_len': original['orig_len'],
    'post_len': postprocessed['post_len'],
    'diff': postprocessed['post_len'] - original['orig_len']
})

# 가장 많이 줄어든 케이스
biggest_reductions = merged.nlargest(10, 'diff', keep='first')
print("⬇️ 가장 많이 변화한 케이스 (Top 10):")
for _, row in biggest_reductions.iterrows():
    print(f"  {row['fname']}: {row['orig_len']:.0f} → {row['post_len']:.0f} ({row['diff']:+.0f} 단어)")

print(f"\n✅ 변화된 케이스: {len(merged[merged['diff'] != 0])}개 / {len(merged)}개")

## 8. 🎉 성공 분석 및 추가 개선 전략

### 📊 실제 결과:
- **원본 (v3)**: 51.7703 (R1=60.01, R2=42.19, RL=53.11)
- **후처리 (v3_postprocessed)**: **51.9393** (R1=60.03, R2=42.89, RL=52.89)
- **개선**: **+0.169점** 🚀

### ✅ 성공 요인 분석:
1. **ROUGE-2 대폭 상승**: 42.19 → 42.89 (+0.70%, **핵심 개선**)
2. **ROUGE-1 미세 상승**: 60.01 → 60.03 (+0.02%)
3. **ROUGE-L 미세 하락**: 53.11 → 52.89 (-0.22%)

### 🔍 왜 효과가 있었나?
- **3문장 제한**: 불필요한 긴 문장 제거로 precision 향상
- **불완전한 문장 제거**: 깨진 문장으로 인한 노이즈 감소
- **따옴표 제거**: 토큰 매칭 정확도 향상
- **ROUGE-2가 크게 올랐다** = 바이그램(2단어 조합) 매칭이 개선됨